In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

spark = SparkSession.builder.appName('Puzzle').getOrCreate()

In [4]:
df = spark.read.csv('data/puzzle_031.csv', header=True, inferSchema=True)
df.show()


+----------+------------+---------------+------+
|        Dt|         CID|TransactionType|Amount|
+----------+------------+---------------+------+
|01-01-2015|  IshaMattoo|             CR| 18000|
|02-01-2015|  IshaMattoo|             DB|  3000|
|03-01-2015|  IshaMattoo|             CR|  8000|
|04-01-2015|  IshaMattoo|             DB|  9000|
|04-01-2015|AvtaarKishan|             CR|  1000|
|05-01-2015|AvtaarKishan|             CR|  6000|
|05-01-2015|AvtaarKishan|             DB|  5000|
+----------+------------+---------------+------+



In [12]:
df.select(
    '*',
    F.sum(F.when(F.col('TransactionType') =='DB', -1*F.col('Amount')
            ).otherwise(F.col('Amount'))
        ).over(W.partitionBy(F.col('CID')).orderBy(F.col('Dt'))).alias('CumSumAmt')
).show()

+----------+------------+---------------+------+---------+
|        Dt|         CID|TransactionType|Amount|CumSumAmt|
+----------+------------+---------------+------+---------+
|04-01-2015|AvtaarKishan|             CR|  1000|     1000|
|05-01-2015|AvtaarKishan|             CR|  6000|     2000|
|05-01-2015|AvtaarKishan|             DB|  5000|     2000|
|01-01-2015|  IshaMattoo|             CR| 18000|    18000|
|02-01-2015|  IshaMattoo|             DB|  3000|    15000|
|03-01-2015|  IshaMattoo|             CR|  8000|    23000|
|04-01-2015|  IshaMattoo|             DB|  9000|    14000|
+----------+------------+---------------+------+---------+

